Create markdown tables for use in SMTN-002

The tables in SMTN-002 include:  
* zeropoints (zp_1s) for the telescope, for each filter.
* skybrightness / skycounts per filter
* FWHMeff fiducial values
* m5 in each bandpass for dark-sky,zenith
* Cm, dCm_infinity, k_atm

In [ ]:
import os
import shutil
import subprocess
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import rubin_sim.phot_utils as phot_utils
import syseng_throughputs as se

Load throughput components up directly from SE 
See github repo: https://github.com/lsst-pst/syseng_throughputs

In [ ]:
defaultDirs = se.setDefaultDirs()
defaultDirs

In [ ]:
atmos = phot_utils.Bandpass()
atmos.read_throughput(os.path.join(defaultDirs['atmosphere'], 'atmos_10_aerosol.dat'))

darksky = phot_utils.Sed()
darksky.read_sed_flambda(os.path.join(defaultDirs['atmosphere'], 'darksky.dat'))

In [ ]:
# Check version information: 
rootDir = defaultDirs['atmosphere'].replace('/siteProperties', '')
gitcommand = f"cd {rootDir} ; git describe"
ret = subprocess.run(gitcommand, capture_output=True, shell=True)
print("Syseng_throughputs repo is at version: ", ret.stdout.decode())

In [ ]:
# Create equivalent of sims syeng_values file .. this should have the Cm/etc values we want. 

addLosses = True
hardware, system = se.buildHardwareAndSystem(defaultDirs, addLosses, atmosphereOverride=atmos)

# --note these are 1x30s and the overview paper uses 2x15s!!! 
# (this is used as the 'base exposure time' in sysEngVals and the m5 calculation from Cm/dCm_infinity)
# Note also this is using fiducial (old) values of the seeing to set the m5 values as well. 
# thus the resulting values are different than those reported in pstn-054, where the Cm/dCm reported values are 
# for the median delivered seeing in the opsim databases.

exptime = 30
nexp = 1
m5_1 = se.makeM5(hardware, system, 
                   exptime={'u': exptime, 'g': exptime, 'r': exptime, 'i': exptime, 'z': exptime, 'y': exptime},
                   nexp={'u': nexp, 'g': nexp, 'r': nexp, 'i': nexp, 'z': nexp, 'y': nexp},
                   readnoise=8.8, othernoise=0, darkcurrent=0.2)

m5_1.round(2)

In [ ]:
# as an example .. if we used 1x30s for u band and 2x15s for the other bands .. 

exptime = 15
nexp = 2
m5_o = se.makeM5(hardware, system, 
                   exptime={'u': 30, 'g': exptime, 'r': exptime, 'i': exptime, 'z': exptime, 'y': exptime},
                   nexp={'u': 1, 'g': nexp, 'r': nexp, 'i': nexp, 'z': nexp, 'y': nexp},
                   readnoise=8.8, othernoise=0, darkcurrent=0.2)

m5_o.round(2)

Ok, so what values should we put into the SMTN-002 technote? 
The technote focuses on 1x30s visits, so let's use those. 

In [ ]:
print(pd.DataFrame(m5_1['Zp_t'].round(2)).to_markdown())

In [ ]:
print(pd.DataFrame(m5_1['skyMag'].round(2)).to_markdown())

In [ ]:
print(pd.DataFrame(m5_1['FWHMeff'].round(2)).to_markdown())

In [ ]:
print(pd.DataFrame(m5_1['m5'].round(2)).to_markdown())

In [ ]:
print(pd.DataFrame(m5_1[['Cm', 'dCm_infinity', 'kAtm']].round(2)).to_markdown())